In [1]:
from lightfm import LightFM

C:\Users\Vansh Narula\AppData\Local\conda\conda\envs\tf_gpu\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
import os
import zipfile
import csv

import requests

def get_data():
    
        return (
            csv.DictReader(x for x in open('new_ratings.csv', newline='')),
            csv.DictReader((x for x in open('embedding.csv', newline=""))),
        )


def get_ratings():

    return get_data()[0]


def get_movie_features():

    return get_data()[1]

In [3]:
import json
from itertools import islice

# with open('train_movie.csv', newline='') as csvfile:
# reader2 = csv.DictReader(x for x in open('train_movie.csv', newline=''))
# movie_features = csv.DictReader((x for x in open('movies_info.csv', newline="")),delimiter="\t")
# for x in movie_features:
#     print(x)

reader,movie_features = get_data()


In [4]:
for line in islice(reader, 2):
    print(json.dumps(line, indent=4))

{
    "user_id": "4",
    "book_id": "70",
    "rating": "4"
}
{
    "user_id": "4",
    "book_id": "264",
    "rating": "3"
}


In [5]:
for line in islice(movie_features, 1):
    print(json.dumps(line, indent=4))

{
    "book_id": "1",
    "embeddings": "[-3.6751375e-02 -2.6640910e-01  1.9817963e-01  2.3172754e-01\r\n  3.2512277e-01 -1.6533449e-01 -6.9004215e-02 -6.2199039e-03\r\n  2.8828798e-02 -4.5724803e-01  6.4509317e-02 -3.6452550e-01\r\n -7.4834406e-02 -3.5423439e-03  1.6090445e-01  1.3766357e-01\r\n  6.6039595e-03 -1.9071519e-02 -5.9100904e-02  1.0679536e-02\r\n  2.0115265e-01  2.3649640e-01  1.4553389e-01  1.2703294e-01\r\n  2.5599179e-01 -2.5997132e-03  1.6416980e-02  7.8967191e-02\r\n -3.9393973e-02 -1.1838774e-01  2.5008342e-01 -3.6555097e-02\r\n -1.7071323e-01  9.8176055e-02  6.3870497e-02 -1.2785308e-01\r\n -1.9390856e-01  2.5614700e-01  4.8297748e-02  7.8030829e-03\r\n -1.9730476e-01 -8.3261371e-02 -1.2594897e-01 -1.9532961e-01\r\n  6.1551351e-02  1.9691898e-01 -3.2243494e-02 -2.0279317e-01\r\n  5.4989889e-02 -1.7494653e-01 -1.3510884e-01  2.5696206e-01\r\n -3.1229377e-01  2.2244476e-01 -6.6466965e-02  3.5466227e-01\r\n  2.5924814e-01  2.6181552e-01  1.5850952e-01  3.1896405e-02\r\

In [6]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit((x['user_id'] for x in get_ratings()),
            (x['book_id'] for x in get_ratings()))

In [7]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 10000, num_items 6176.


In [8]:
dataset.fit_partial(items=(x['book_id'] for x in get_movie_features()),
                    item_features=(x['embeddings'] for x in get_movie_features()))

In [10]:
# dataset.fit_partial(items=(x['book_id'] for x in get_movie_features()),item_features=(x['title'] for x in get_movie_features()))

In [9]:
(interactions, weights) = dataset.build_interactions((x['user_id'], x['book_id'], float(x['rating']))
                                                      for x in get_ratings())


In [10]:
item_features = dataset.build_item_features(((x['book_id'], [x['embeddings']])
                                              for x in get_movie_features()))
print(repr(item_features))

<6176x12352 sparse matrix of type '<class 'numpy.float32'>'
	with 12352 stored elements in Compressed Sparse Row format>


In [11]:
from lightfm import LightFM
model = LightFM(learning_rate=0.01, loss='bpr', no_components = 20)

model.fit(interactions, item_features=item_features, epochs=500, num_threads=4)

In [ ]:
from lightfm import LightFM
model = LightFM(learning_rate=0.01, loss='bpr', no_components = 20)

model.fit(interactions, epochs=500)

In [17]:
from lightfm.evaluation import precision_at_k
train_precision = precision_at_k(model, interactions, k=5).mean()
print(train_precision)

0.75752


In [14]:
from lightfm.evaluation import precision_at_k,auc_score
train_precision = precision_at_k(model, interactions,item_features=item_features, k=5).mean()
print(train_precision)

0.76334006


In [18]:
train_auc = auc_score(model, interactions,item_features=item_features).mean()
print(train_auc)

0.89994687
